# Gold Layer: Analytics & Aggregations

## Purpose
Read cleaned data from the Silver Delta table and produce aggregated analytics
fact tables for dashboards, reporting, and alerting.

## Data Flow
```
Silver Delta Table (Clean) -> Aggregate -> Gold Fact Tables (Analytics)
```

## Outputs
| Table | Description |
|---|---|
| `gold_fraud_summary` | Fraud vs Legitimate totals and Amount stats |
| `gold_hourly_patterns` | Hourly transaction counts with watermark |
| `gold_high_value_alerts` | High-value alerts with severity levels |

## 1. Cleanup (Optional)

Run this cell to **reset the Gold tables and checkpoints** before a fresh start.
Skip this cell if you want to keep existing data.

In [0]:
# 1. Stop active streams
for s in spark.streams.active:
    s.stop()
print("All active streams stopped.")

# 2. Drop the Gold summary table
try:
    spark.sql("DROP TABLE IF EXISTS fraud_lakehouse_workspace.default.gold_fraud_summary")
    print("Gold summary table dropped.")
except Exception as e:
    print(f"Failed to drop Gold summary table: {e}")

# 3. Delete the Gold checkpoint
gold_checkpoint = "YOUR_CHECKPOINT_PATH_HERE" # Replace with your actual checkpoint path, e.g., "dbfs:/fraud_lakehouse_workspace/checkpoints/gold_summary_checkpoint"
try:
    dbutils.fs.rm(gold_checkpoint, recurse=True)
    print("Gold checkpoint cleared.")
except Exception as e:
    print(f"Failed to clear Gold checkpoint: {e}")

print("Gold slate is clean. Ready to rebuild analytics.")

## 2. Gold Layer - Build All Fact Tables

This cell builds three Gold analytics tables from the Silver stream:

| Table | Output Mode | Description |
|---|---|---|
| `gold_fraud_summary` | Complete | Fraud vs Legitimate totals, avg/min/max/sum of Amount |
| `gold_hourly_patterns` | Append | Hourly transaction counts and amounts with watermark |
| `gold_high_value_alerts` | Append | Alerts for transactions over 1000 with severity |

In [0]:
from pyspark.sql.functions import *

# Define checkpoint paths
gold_checkpoint_summary = "YOUR_CHECKPOINT_PATH_HERE" # Replace with your actual checkpoint path, e.g., "dbfs:/fraud_lakehouse_workspace/checkpoints/gold_summary_checkpoint"
gold_checkpoint_hourly = "YOUR_CHECKPOINT_PATH_HERE" # Replace with your actual checkpoint path, e.g., "dbfs:/fraud_lakehouse_workspace/checkpoints/gold_hourly"
gold_checkpoint_alerts = "YOUR_CHECKPOINT_PATH_HERE" # Replace with your actual checkpoint path, e.g., "dbfs:/fraud_lakehouse_workspace/checkpoints/gold_alerts"

print("Building Gold Layer (Multiple Fact Tables)...")

# Read from Silver
try:
    df_silver_stream = spark.readStream \
        .table("fraud_lakehouse_workspace.default.silver_transactions")
    print("Connected to Silver stream.")
except Exception as e:
    print(f"Failed to connect to Silver stream: {e}")
    raise

# ========================================
# GOLD TABLE 1: Fraud Summary by Class
# ========================================
try:
    df_gold_summary = df_silver_stream \
        .groupBy("Class") \
        .agg(
            count("*").alias("total_transactions"),
            avg("Amount").alias("avg_amount"),
            min("Amount").alias("min_amount"),
            max("Amount").alias("max_amount"),
            sum("Amount").alias("total_amount")
        )

    query_summary = df_gold_summary.writeStream \
        .format("delta") \
        .outputMode("complete") \
        .option("checkpointLocation", gold_checkpoint_summary) \
        .toTable("fraud_lakehouse_workspace.default.gold_fraud_summary")

    print("Gold Summary table streaming!")
except Exception as e:
    print(f"Gold Summary table failed: {e}")
    raise

# ========================================
# GOLD TABLE 2: Hourly Transaction Patterns (WITH WATERMARK)
# ========================================
try:
    df_gold_hourly = df_silver_stream \
        .withWatermark("silver_processed_time", "10 minutes") \
        .groupBy(
            col("transaction_date"),
            col("transaction_hour"),
            col("Class"),
            window(col("silver_processed_time"), "1 hour")
        ) \
        .agg(
            count("*").alias("transaction_count"),
            avg("Amount").alias("avg_amount"),
            sum("Amount").alias("total_amount"),
            sum("is_high_value").alias("high_value_count")
        ) \
        .select(
            col("transaction_date"),
            col("transaction_hour"),
            col("Class"),
            col("window.start").alias("window_start"),
            col("window.end").alias("window_end"),
            col("transaction_count"),
            col("avg_amount"),
            col("total_amount"),
            col("high_value_count")
        ) \
        .withColumn("gold_created_time", current_timestamp())

    query_hourly = df_gold_hourly.writeStream \
        .format("delta") \
        .outputMode("append") \
        .option("checkpointLocation", gold_checkpoint_hourly) \
        .toTable("fraud_lakehouse_workspace.default.gold_hourly_patterns")

    print("Gold Hourly Patterns table streaming!")
except Exception as e:
    print(f"Gold Hourly Patterns table failed: {e}")
    raise

# ========================================
# GOLD TABLE 3: High-Value Alerts (NO AGGREGATION - NO WATERMARK NEEDED)
# ========================================
try:
    df_gold_alerts = df_silver_stream \
        .filter(col("is_high_value") == 1) \
        .select(
            col("Time"),
            col("transaction_date"),
            col("transaction_hour"),
            col("Amount"),
            col("Class"),
            col("is_fraud"),
            col("amount_category"),
            col("silver_processed_time")
        ) \
        .withColumn("alert_severity",
                    when(col("Amount") > 5000, "Critical")
                    .when(col("Amount") > 2000, "High")
                    .otherwise("Medium"))

    query_alerts = df_gold_alerts.writeStream \
        .format("delta") \
        .outputMode("append") \
        .option("checkpointLocation", gold_checkpoint_alerts) \
        .toTable("fraud_lakehouse_workspace.default.gold_high_value_alerts")

    print("Gold Alerts table streaming!")
except Exception as e:
    print(f"Gold Alerts table failed: {e}")
    raise

print("All Gold tables are streaming!")
print("Tables created:")
print("  1. gold_fraud_summary (complete mode)")
print("  2. gold_hourly_patterns (append mode with watermark)")
print("  3. gold_high_value_alerts (append mode, no aggregation)")

## 3. Alternative: Simplified Gold Summary (Optional)

A simplified version that creates only the fraud summary table with basic aggregations.
Use this for testing or if the full Gold pipeline above encounters issues.

> **Note:** Only run this if the full pipeline above has not been started,
> or after running the cleanup cell first.

In [0]:
from pyspark.sql.functions import col, count, avg

try:
    # 1. Read from Silver table
    df_silver_stream = spark.readStream \
        .table("fraud_lakehouse_workspace.default.silver_transactions")

    # 2. Aggregate
    df_gold_summary = df_silver_stream \
        .groupBy("Class") \
        .agg(
            count("*").alias("Total_Transactions"),
            avg("Amount").alias("Average_Amount")
        )

    # 3. Write Stream (Complete mode is best for small aggregations)
    gold_query = df_gold_summary.writeStream \
        .format("delta") \
        .outputMode("complete") \
        .option("checkpointLocation", gold_checkpoint) \
        .toTable("fraud_lakehouse_workspace.default.gold_fraud_summary")

    print("Simplified Gold aggregation is running on clean data!")
except Exception as e:
    print(f"Simplified Gold summary failed: {e}")
    raise

## 4. Verify: Gold Summary Data

Query the Gold summary table to confirm fraud and legitimate transaction
counts and amounts are being aggregated correctly.

In [0]:
try:
    display(spark.sql("SELECT * FROM fraud_lakehouse_workspace.default.gold_fraud_summary"))
except Exception as e:
    print(f"Failed to query Gold summary: {e}")
    raise

## Summary

**Gold Layer Status:**
- Read from Silver Delta table (streaming)
- `gold_fraud_summary` - Aggregated by Class with count, avg, min, max, sum of Amount
- `gold_hourly_patterns` - Hourly patterns with 10-minute watermark and 1-hour window
- `gold_high_value_alerts` - Alerts for high-value transactions with Critical / High / Medium severity

**Pipeline Complete:** Bronze -> Silver -> Gold